# Collect demonstrations

Interactive notebook for collecting demonstrations

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import json
import random
from datetime import datetime
from demonstration_recorder import DemonstrationRecorder

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxend module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot
available_robot = False
if available_robot:
    from robot.al5d_position_controller import PositionController, RobotPosition

from camera.camera_controller import CameraController
from demonstration import Demonstration


Approxend module not found, cannot use gamepad


In [4]:
experiment = "demonstration_collector"
run = "collect_freeform"

exp = Config().get_experiment(experiment, run)
exp_target = Config().get_experiment("demonstration", exp["demonstration_name"])
exp_robot_controller = Config().get_experiment(exp["exp_robot_controller"], exp["run_robot_controller"])
exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])
exp_keyboard_controller = Config().get_experiment(exp["exp_keyboard_controller"], exp["run_keyboard_controller"])
exp_gamepad_controller = Config().get_experiment(exp["exp_gamepad_controller"], exp["run_gamepad_controller"])

***ExpRun**: Configuration for exp/run: demonstration_collector/collect_freeform successfully loaded
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\demonstration\_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\controllers\_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/robot_position_controller successfully loaded
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\controllers\_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_cam0_controller successfully loaded
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experime

In [5]:
# starting the robot controller
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None
# starting the camere controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True

# find the demonstration path
# _, task_dir = ui_choose_task(offer_task_creation=True)

task_dir = exp_target.data_dir()
demoname = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
demo_dir = pathlib.Path(task_dir, demoname)
demo_dir.mkdir(parents=False, exist_ok=False)
print(f"Ok, demonstration with go into\n\t {demo_dir}")

demonstration = Demonstration(exp_target, demoname, parse_old_style = False)
demonstration.metadata["cameras"] = sorted(camera_controller.capture_devs.keys())
demonstration.save_metadata()
# fixme need to add the cameras into the demonstration

cap0 works
Ok, demonstration with go into
	 c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\demonstration\freeform\2025_06_28__10_48_05


In [4]:
# Performs the recording. The robot is controlled through the primary 
# controller. This is also used to terminate the collection
# The information is collected by the demonstration recorder

controller = exp["primary_controller"]

if controller == "xbox":
    gamepad_controller = GamepadController(exp_gamepad_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration,
        camera_controller=camera_controller, remote_control=gamepad_controller, robot_controller=
                                robot_controller, save_dir=demo_dir, task_name=task_dir.name)
    # dr = None
    gamepad_controller.demonstration_recorder = demo_recorder
    gamepad_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")
if controller == "keyboard":
    kb_controller = KeyboardController(exp_keyboard_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=kb_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    kb_controller.demonstration_recorder = demo_recorder
    kb_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "program":
    program_controller = ProgramController(
        robot_controller=robot_controller, camera_controller=camera_controller)
    
    # create wpcount 
    waypoints = []
    wpcount = 10
    while True:
        norm = [0] * 6
        for df in range(6):
            norm[df] = random.random()                
        rp = RobotPosition.from_normalized_vector(norm)
        if RobotPosition.limit(rp):
            print(f"added waypoint {rp}")
            waypoints.append(rp)
        if len(waypoints) >= wpcount:
            break

    program_controller.waypoints = waypoints
    program_controller.interactive_confirm = False


    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=program_controller, robot_controller=
                                robot_controller, save_dir=demo_dir, task_name=task_dir.name)

    program_controller.demonstration_recorder = demo_recorder
    demonstration.save_metadata()
    program_controller.control()


INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


-1
-1
-1


INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


-1
-1


INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


...

-1
-1


INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


-1
-1


INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


-1
====== Demonstration terminated and recorded successfully, bye. ======


In [5]:
demonstration.save_metadata()

In [6]:
print(demonstration.metadata)

{'stored_as_video': False, 'stored_as_images': True, 'maxsteps': 106, 'cameras': ['dev0']}
